## Spatial Prediction Skill Evaluation of DPLE FOSI hindcast

In [1]:
import xarray as xr
import numpy as np
import os
import pandas as pd
from scipy.stats import pearsonr
import tools
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
datadir = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
var = 'Phi'
tres = 'yearly'

## DPLE FOSI hindcast 1948-2017

In [3]:
# original FOSI hindcast as predictand
filename = f'{datadir}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_{tres}.nc'
ds_hindo = xr.open_dataset(filename)
lon = ds_hindo['TLONG'].values - 360.
lat = ds_hindo['TLAT'].values
nlat = ds_hindo.dims['nlat']
nlon = ds_hindo.dims['nlon']

In [4]:
# original FOSI hindcast as persistence forecast
filename = f'{datadir}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_{tres}.nc'
ds_hind = xr.open_dataset(filename)

## DPLE forecast ensemble mean 1954-2017

In [5]:
# DPLE forecasts
filename = f'{datadir}/DPLE_driftcorrected_Phi_Eo_space_vertical_mean_ens_mean_{tres}.nc'
ds_fore = xr.open_dataset(filename)

### Layer Selection

In [6]:
#layer = '0-200m'
layer = '200-600m'

## Cycle over spatial values for lead year ACC

In [7]:
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
#
alpha = 0.05
# lead year range: 1,2,...,9,10
ldyrs = range(1, 11)
#ld1 = 1; ld2 = 5 # 1-5 yr prediction
#ld1 = 3; ld2 = 7 # 3-7 yr prediction
#ld1 = 5; ld2 = 9 # 5-9 yr prediction
# we only calculate 1 Eo species: Eo=0.4 eV
vv = 6
corr_p = np.ma.zeros([len(ldyrs), a2-a1, b2-b1]); corr_p.mask=True
rcri_p = np.ma.zeros([len(ldyrs), a2-a1, b2-b1]); rcri_p.mask=True
corr_d = corr_p.copy()
rcri_d = rcri_p.copy()
corr13 = corr_p.copy()
rcri13 = rcri_p.copy()
nmae_p = corr_p.copy()
nmae_d = corr_p.copy()
#
for ld1, hh in zip(ldyrs, range(len(ldyrs))):
        ld2 = ld1 #+2
        # persistence ACC: var1; var2
        # DPLE ACC:        var2; var3
        var1 = ds_hind[f'{var}_{vv}_{layer}'][yr1-1948:yr2-1948+1].values
        var2 = 0.
        var3 = 0.
        for lead in range(ld1, ld2+1):
            var2 = var2 + ds_hindo[f'{var}_{vv}_{layer}'][yr1-1948+lead:yr2-1948+1+lead].values
            var3 = var3 + ds_fore[f'{var}_{vv}_{layer}'][yr1-1954:yr2-1954+1, lead-1].values
        #
        var2 = var2 / np.float(ld2+1-ld1)
        var3 = var3 / np.float(ld2+1-ld1)
        # mask nan values
        mask1 = np.isnan(var1)
        mask2 = np.isnan(var2)
        mask3 = np.isnan(var3)
        var1 = np.ma.array(var1, mask=mask1)
        var2 = np.ma.array(var2, mask=mask2)
        var3 = np.ma.array(var3, mask=mask3)
        ##
        for ii in range(a2-a1):
            for jj in range(b2-b1):
                # persistence ACC: var1;var2
                # DPLE ACC:        var2;var3
                # var1; var3
                mem1 = var1[:, ii, jj]
                mem2 = var2[:, ii, jj]
                mem3 = var3[:, ii, jj]
                ### Persistence ACC & NMAE
                fmask = np.logical_and(mem1.mask, mem2.mask)
                if np.sum(fmask) != len(fmask):
                    nen1 = mem1[~fmask]
                    nen2 = mem2[~fmask]
                    corr_p[hh, ii, jj] = pearsonr(nen1, nen2)[0]
                    neff = tools.neff3(nen1, nen2, len(nen1))
                    rcri_p[hh, ii, jj] = tools.calculate_parson_corr_critical_value(neff, alpha)
                    #
                    nen1 = nen1 - np.nanmean(nen1)
                    nen2 = nen2 - np.nanmean(nen2)
                    #rmse_p[cc, hh, ii, jj] = np.sqrt(np.nanmean((nen1-nen2)**2))
                    nmae_p[hh, ii, jj] = np.nanmean(np.abs(nen1-nen2)/np.nanstd(nen2))
                ### DPLE ACC & NMAE
                fmask = np.logical_and(mem2.mask, mem3.mask)
                if np.sum(fmask) != len(fmask):
                    nen2 = mem2[~fmask]
                    nen3 = mem3[~fmask]
                    corr_d[hh, ii, jj] = pearsonr(nen2, nen3)[0]
                    neff = tools.neff3(nen2, nen3, len(nen2))
                    rcri_d[hh, ii, jj] = tools.calculate_parson_corr_critical_value(neff, alpha)
                    #
                    nen3 = nen3 - np.nanmean(nen3)
                    nen2 = nen2 - np.nanmean(nen2)
                    #rmse_d[cc, hh, ii, jj] = np.sqrt(np.nanmean((nen2-nen3)**2))
                    nmae_d[hh, ii, jj] = np.nanmean(np.abs(nen3-nen2)/np.nanstd(nen2))
                ### Correlation for difference significance test
                fmask = np.logical_and(mem1.mask, mem3.mask)
                if np.sum(fmask) != len(fmask):
                    nen1 = mem1[~fmask]
                    nen3 = mem3[~fmask]
                    corr13[hh, ii, jj] = pearsonr(nen1, nen3)[0]
                    neff = tools.neff3(nen1, nen3, len(nen1))
                    rcri13[hh, ii, jj] = tools.calculate_parson_corr_critical_value(neff, alpha)

In [8]:
data = xr.Dataset({"corr_p": (("nlead", "nlat", "nlon"), corr_p),\
                   "rcri_p": (("nlead", "nlat", "nlon"), rcri_p),\
                   "corr_d": (("nlead", "nlat", "nlon"), corr_d),\
                   "rcri_d": (("nlead", "nlat", "nlon"), rcri_d),\
                   "corr13": (("nlead", "nlat", "nlon"), corr13),\
                   "rcri13": (("nlead", "nlat", "nlon"), rcri13),\
                   "nmae_p": (("nlead", "nlat", "nlon"), nmae_p),\
                   "nmae_d": (("nlead", "nlat", "nlon"), nmae_d),\
                   "lon": (("nlat", "nlon"), lon),\
                   "lat": (("nlat", "nlon"), lat)})

In [9]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:  (nlead: 10, nlat: 140, nlon: 195)
Dimensions without coordinates: nlead, nlat, nlon
Data variables:
    corr_p   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    rcri_p   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    corr_d   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    rcri_d   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    corr13   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    rcri13   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    nmae_p   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    nmae_d   (nlead, nlat, nlon) float64 nan nan nan nan nan ... nan nan nan nan
    lon      (nlat, nlon) float64 -258.8 -257.7 -256.6 ... -41.41 -40.84 -40.28
    lat      (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42

In [10]:
data.to_netcdf(f'{dout}/DPLE_spatial_persistence_DPLE_ACC_NMAE_medium_Eo_{var}_{layer}_leadyear1-10.nc', mode='w')